In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline

plt.rcParams['figure.dpi'] = 100

In [5]:
def access_covid_api():
    filters=['areaType=overview']
    structure={"date":"date",
        "firstVac":"cumPeopleVaccinatedFirstDoseByPublishDate",
        "secondVac":"cumPeopleVaccinatedSecondDoseByPublishDate"}
    api=Cov19API(filters,structure)
    data=api.get_json()
    with open('vacTimes.json','wt') as OUTF:
        json.dump(data, OUTF)
    with open('vacTimes.json','rt') as INFILE:    
        vacData=json.load(INFILE)
    return vacData


vacData=access_covid_api()  #it works
vacData

{'data': [{'date': '2021-12-08', 'firstVac': 51183457, 'secondVac': 46640237},
  {'date': '2021-12-07', 'firstVac': 51161757, 'secondVac': 46610800},
  {'date': '2021-12-06', 'firstVac': 51138245, 'secondVac': 46582425},
  {'date': '2021-12-05', 'firstVac': 51118266, 'secondVac': 46557413},
  {'date': '2021-12-04', 'firstVac': 51096311, 'secondVac': 46530247},
  {'date': '2021-12-03', 'firstVac': 51069397, 'secondVac': 46492449},
  {'date': '2021-12-02', 'firstVac': 51046133, 'secondVac': 46462638},
  {'date': '2021-12-01', 'firstVac': 51020285, 'secondVac': 46431662},
  {'date': '2021-11-30', 'firstVac': 50994257, 'secondVac': 46399306},
  {'date': '2021-11-29', 'firstVac': 50963718, 'secondVac': 46367149},
  {'date': '2021-11-28', 'firstVac': 50941327, 'secondVac': 46341057},
  {'date': '2021-11-27', 'firstVac': 50919834, 'secondVac': 46311612},
  {'date': '2021-11-26', 'firstVac': 50896026, 'secondVac': 46278746},
  {'date': '2021-11-25', 'firstVac': 50875712, 'secondVac': 46255081}

In [4]:
def parse_date(date_string):
    return pd.to_datetime(date_string,format="%Y-%m-%d")
def wrangle_data(olddata):
    datalist=olddata['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate,enddate,freq='D')
    vacTimesdf=pd.DataFrame(index=index,columns=['firstVac','secondVac'])
    for line in datalist:
        data=parse_date(line['date'])
        for column in ['firstVac','secondVac']:
            value=float(line[column] if line[column] !=None else 0.0)
            vacTimesdf.loc[data,column]=value
            
    vacTimesdf.fillna(0.0,inplace=True)
    return vacTimesdf

vacTimesdf=wrangle_data(vacData)

In [5]:
vac_clos=wdg.SelectMultiple(
    options=['firstVac','secondVac'],
    value=['firstVac','secondVac'],
    rows=3,
    description='Vaccinated:',
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.VBox([vac_clos, scale])
#plt.ticklabel_format(style='plain')
def vac_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        vacTimesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(vac_graph, {'gcols': vac_clos, 'gscale': scale})

display(controls, graph)

Output()

In [6]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    vacData=access_covid_api(filters,structure)
    # wrangle the data and overwrite the dataframe for plotting
    global vacTimesdf
    vacTimesdf=wrangle_data(vacData)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    #refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='PANIC', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

Button(button_style='danger', description='PANIC', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…